In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
train_set="/kaggle/input/customer-segmentation/Train.csv"
# train_set="Train.csv"
test_set="/kaggle/input/customer-segmentation/Test.csv"

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv(train_set)
datan=pd.read_csv(test_set)
data=pd.concat([data,datan],axis=0)
del data['ID']
data.head()

In [ ]:
print("Number of null values for each feature:")
print(data.shape)
print(np.sum(data.isna()))
data = data.dropna()
print(data.shape)

### Features
1. Gender -> Nominal
2. Ever_Married -> Nominal
3. Age -> Absolute
4. Graduated -> Nominal
5. Profession -> Nominal
6. Work_Experience -> Absolute
7. Spending_Score -> Ordinal
8. Family_Size -> Absolute
9. Var_1 -> Nominal

In [ ]:
sns.histplot(data['Age'], kde=True, color='darkblue')
plt.title('Age')
plt.show()
sns.histplot(data['Work_Experience'], kde=True, color='darkblue', binwidth=1)
plt.title('Work Experience')
plt.show()
sns.histplot(data['Family_Size'], kde=True, color='darkblue', binwidth=1, kde_kws={'gridsize': 10})
plt.title('Family Size')
plt.show()

In [ ]:
data['Profession'].value_counts().plot(kind='bar')
plt.title('Profession')
plt.show()
data['Spending_Score'].value_counts().plot(kind='bar')
plt.title('Spending Score')
plt.show()
data['Gender'].value_counts().plot(kind='bar')
plt.title('Gender')
plt.show()
data['Var_1'].value_counts().plot(kind='bar')
plt.title('Anonymous Category')
plt.show()

In [ ]:
plt.boxplot(data['Age'], vert=False)
plt.subplots_adjust(bottom=0.25)
plt.title('Age')
plt.show()

plt.boxplot(data['Work_Experience'], vert=False)
plt.subplots_adjust(bottom=0.25)
plt.title('Work Experience')
plt.show()

plt.boxplot(data['Family_Size'], vert=False)
plt.subplots_adjust(bottom=0.25)
plt.title('Family Size')
plt.show()

In [ ]:

data=data.reset_index(drop=True)
new_data = data
print(new_data)
print(new_data.shape)
new_data = new_data.replace({'Spending_Score': {'Low': 1, 'Average': 2, 'High': 3}})
new_data = new_data.replace({'Ever_Married': {'Yes': 1, 'No': 0}})
new_data = new_data.replace({'Gender': {'Male': 1, 'Female': 0}})
new_data = new_data.replace({'Graduated': {'Yes': 1, 'No': 0}})
print(new_data.head())

In [ ]:
stats = new_data.describe()
stats.loc['IQR'] = stats.loc['75%'] - stats.loc['25%'] # appending interquartile range instead of recalculating it
stats = stats.append(new_data.agg(['skew', 'kurt', 'median']))
print(stats)


In [ ]:
print(new_data.corr())

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
encoded_data = new_data
# print(new_data)
enc_df1 = pd.DataFrame(enc.fit_transform(encoded_data[['Profession','Var_1']]).toarray())
# print(enc_df)
encoded_data = pd.concat([encoded_data, enc_df1], axis=1)
# # print(encoded_data)
# enc_df2 = pd.DataFrame(enc.fit_transform(encoded_data[['Var_1']]).toarray())
# encoded_data = pd.concat([encoded_data,enc_df1, enc_df2], axis=1)
# # print(new_data.shape)

del encoded_data['Profession']
del encoded_data['Var_1']
segmentations=encoded_data['Segmentation']
del encoded_data['Segmentation']
print(encoded_data.head())
# print(new_data.head())
# print(np.sum(encoded_data.isna()))
# enc_df.isna().any()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_data = encoded_data.copy()

scaled_data[['Age', 'Work_Experience', 'Spending_Score', 'Family_Size']] = scaler.fit_transform(scaled_data[['Age', 'Work_Experience', 'Spending_Score', 'Family_Size']])
print(encoded_data)
print(scaled_data.head())

In [ ]:
# print(np.sum(scaled_data.isna()))
# from sklearn.decomposition import PCA
# pca = PCA(n_components=2)
# principalComponents = pca.fit_transform(scaled_data)
# principalDf = pd.DataFrame(data = principalComponents
#              , columns = ['principal component 1', 'principal component 2'])

In [ ]:
from sklearn.cluster import KMeans
wcss=[]
for i in range(1,11):
    kmeans = KMeans(i)
    kmeans.fit(scaled_data)
    wcss_iter = kmeans.inertia_
    wcss.append(wcss_iter)

number_clusters = range(1,11)
plt.plot(number_clusters,wcss)
plt.title('The Elbow title')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')

In [ ]:
kmean=KMeans(4,random_state=0)
kmean.fit(scaled_data)

In [ ]:
identified_clusters = kmean.fit_predict(scaled_data)

In [ ]:
# print(encoded_data)
dicts={'A':1,'B':0,'C':2,'D':3}
ttt=np.array(segmentations.values)
print(type(ttt))
nar=np.zeros(len(ttt))
for i in range(len(ttt)):
    #print(ttt[i])
    nar[i]=dicts[ttt[i]]
# segmentations=dicts[ttt]


print(identified_clusters[:20])
# for i in range(10):
kmean=KMeans(4)
kmean.fit(scaled_data)
identified_clusters = kmean.fit_predict(scaled_data)
print(np.sum(nar==identified_clusters))

idx0=np.where(identified_clusters==0)
idx1=np.where(identified_clusters==1)
idx2=np.where(identified_clusters==2)
idx3=np.where(identified_clusters==3)

print(new_data[:].iloc[idx0].describe())
print(new_data[:].iloc[idx1].describe())
print(new_data[:].iloc[idx2].describe())
print(new_data[:].iloc[idx3].describe())

# data_with_clusters = scaled_data.copy()
# data_with_clusters['Clusters'] = identified_clusters 
# plt.scatter(data_with_clusters['Age'],data_with_clusters['Family_Size'],c=data_with_clusters['Clusters'],cmap='rainbow')

#D 3 B 0 C 2 A 1

In [ ]:
from sklearn.model_selection import train_test_split

seed=450

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier
from xgboost import XGBClassifier


lr = LogisticRegression(n_jobs = -1)
svc = SVC() 
knn = KNeighborsClassifier(n_jobs= -1)
rf = RandomForestClassifier(n_jobs = -1)
xgb = XGBClassifier(n_jobs = -1)
gb=GradientBoostingClassifier()
abc=AdaBoostClassifier()


In [ ]:
# d_train=pd.concat([scaled_data,segmentations],axis=1)
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
segs=segmentations
segs = labelencoder.fit_transform(segs)


# print(d_train)
X= scaled_data
Y= segs

print(X.head())
print(Y)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)

In [62]:
def fit_model(model):
    model.fit(X_train, y_train)
    return (model.score(X_train, y_train),model.score(X_test, y_test))
    
models = {'LogisticRegression': lr,
          'KNeighborsClassifier': knn,
          'SVC': svc,
          'RandomForestClassifier': rf,
          'XGBClassifier': xgb,
          'GradientBoostingClassifier': gb,
          'AdaBoostClassifier': abc}
test_scores = {}
train_scores = {}
for index, model in models.items():
    test_scores[index], train_scores[index] = fit_model(model)
test_scores.items()

In [63]:
test_score_backup = test_scores
train_score_backup = train_scores


In [68]:
test_scores = test_score_backup
train_scores = train_score_backup
test_scores = pd.DataFrame(test_scores,index=['Test Score']).transpose()
test_scores = test_scores.sort_values('Test Score')

train_scores = pd.DataFrame(train_scores,index=['Train Score']).transpose()
train_scores = train_scores.sort_values('Train Score')

In [69]:
print(test_scores)
print(train_scores)